<a href="https://colab.research.google.com/github/NanaAkwasiAbayieBoateng/TidyMachineLearning/blob/master/TidyMachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# activate R magic
import rpy2
%load_ext rpy2.ipython

In [0]:
#%%capture  #hide cell output
%%R
install.packages("pacman")

In [0]:
%%R

pacman::p_load(tidyverse,rio,tidymodels)
    

In [0]:
%%capture
%%R

pacman::p_load(skimr,kableExtra,caret,recipes,rsample,yardstick,pROC,rio,
               xgboost,mlr,Hmisc,MLmetrics,data.table,furrr)
    
    
#Load  variable importance plot
source("https://raw.githubusercontent.com/NanaAkwasiAbayieBoateng/MLTools/master/Varplot.R")
source("https://raw.githubusercontent.com/NanaAkwasiAbayieBoateng/MLTools/master/EvaluationMetrics.R")


theme_set(theme_pubclean())




#### XGBoost Grid  Hyperparameter Optimization

Load the data, included in the {mlrbench} package. The original data has the target variable cmedv is a continuous variable. It is converted to a binary variable here based on below and above its median value. This was done because we are interested in training a classification model. The data is included in the mlbench library.

#### Description of Data
The data is available on kaggle. The link to is [here](https://www.kaggle.com/patelprashant/employee-attrition)

- Context
The key to success in any organization is attracting and retaining top talent. I’m an HR analyst at my company, and one of my tasks is to determine which factors keep employees at my company and which prompt others to leave. I need to know what factors I can change to prevent the loss of good people. Watson Analytics is going to help.

- Content
I have data about past and current employees in a spreadsheet on my desk top. It has various data points on our employees, but I’m most interested in whether they’re still with my company or whether they’ve gone to work somewhere else. And I want to understand how this relates to workforce attrition.

- Education
1 'Below College' 2 'College' 3 'Bachelor' 4 'Master' 5 'Doctor'

- EnvironmentSatisfaction
1 'Low' 2 'Medium' 3 'High' 4 'Very High'

- JobInvolvement
1 'Low' 2 'Medium' 3 'High' 4 'Very High'

- JobSatisfaction
1 'Low' 2 'Medium' 3 'High' 4 'Very High'

- PerformanceRating
1 'Low' 2 'Good' 3 'Excellent' 4 'Outstanding'

-- RelationshipSatisfaction
1 'Low' 2 'Medium' 3 'High' 4 'Very High'

- WorkLifeBalance
1 'Bad' 2 'Good' 3 'Better' 4 'Best'

Acknowledgements
https://www.ibm.com/communities/analytics/watson-analytics-blog/watson-analytics-use-case-for-hr-retaining-valuable-employees/


In [0]:
%%R


path = "https://raw.githubusercontent.com/NanaAkwasiAbayieBoateng/Datasets/master/EmployeeAttrition.csv"

employee_data  = rio::import(path)


In [0]:
%%R

install.packages("gtsummary")
install.packages("gt")
install.packages("kable")
install.packages("kableExtra")
#install.packages("glue")
devtools::install_github("tidyverse/glue")
install.packages("mlr")

In [0]:
%%R
#library(gtsummary)
#library(gt)
# make dataset with a few variables to summarize
#trial2 <- trial %>% dplyr::select(trt, age, grade, response)

# summarize the data with our package
#table1 <- tbl_summary(employee_data)

#tbl_summary(employee_data)

In [0]:
%%R
pacman::p_load(kable,kableExtra)


# Perform a quick summary with skim()
#employee_data %>%  skimr::skim() 


skimr::skim(employee_data)%>%
  kable() %>%
  kable_styling()

In [0]:
%%R
 mlr::summarizeColumns(employee_data)%>%  
  
  kable(escape = F, align = "c") %>%
  kable_styling(c("striped", "condensed"), full_width = F)

#### DATA SPLITTING 
The dataset would be split into traning and test sets using the rsample package. The training set is used to train the model and test set would be used for model validation and performance measures.

The rsample package is used to  create an object that contains the information on how to split the data, and then two more rsample functions to create data frames for the training and testing sets: measure.


In [0]:
%%R

train_test_split <- rsample::initial_split(boston3,strata = "price", prop = 0.7)
housing_train <- training(train_test_split)
housing_test <- testing(train_test_split)


In [0]:
%%R

validation_data <- mc_cv(housing_train, prop = 0.9, times = 30)


#### CREATE RECIPE AND ROLES ︎
To get started, let’s create a recipe for an xgboost  model. Before training the model, we can use a recipe to create a few new predictors and conduct some preprocessing required by the model.



In [0]:
%%R

prep_recipe <- function(dataset){
    recipe(price ~ ., data = dataset) %>%
     #   step_center(all_numeric()) %>%
     #   step_scale(all_numeric()) %>%
        step_dummy(all_nominal(),-all_outcomes())%>%
    prep(data = dataset,retain = TRUE )
}



recipe_prepped <- prep_recipe(dataset = housing_train)

train_data <- bake(recipe_prepped, new_data = housing_train)
test_data  <- bake(recipe_prepped, new_data = housing_test)



In [0]:
%%R


my_xgb <- function(mtry, trees,min_n,tree_depth,learn_rate, 
                   loss_reduction,sample_size, split, id){
    analysis_set <- analysis(split)
     #analysis_prep <- prep(simple_recipe(analysis_set), training = analysis_set)
    analysis_prep <- prep_recipe(analysis_set)
    analysis_processed <- bake(analysis_prep, new_data = analysis_set)
    #parsnip::xgb_train()
   model <- boost_tree(mode = "classification", mtry = mtry, trees = trees,
  min_n = min_n, tree_depth = tree_depth, learn_rate = learn_rate,
  loss_reduction = loss_reduction, sample_size = sample_size)%>%
      set_engine("xgboost") %>%
        fit(price ~ ., data = analysis_processed)
    #model <- rand_forest(mode = "regression",mtry = mtry, trees = trees) %>%
    #    set_engine("ranger", importance = 'impurity') %>%
     #   fit(price ~ ., data = analysis_processed)
    assessment_set <- assessment(split)
    assessment_prep <- prep(prep_recipe(assessment_set), testing = assessment_set)
    assessment_processed <- bake(assessment_prep, new_data = assessment_set)
    tibble::tibble("id" = id,
        "truth" = assessment_processed$price,
        "prediction" = unlist(predict(model, new_data = assessment_processed)))
}




my_xgb_reg <- function(mtry, trees,min_n,tree_depth,learn_rate, 
                   loss_reduction,sample_size, split, id){
     analysis_set <- analysis(split)
    #analysis_prep <- prep(simple_recipe(analysis_set), training = analysis_set)
    analysis_prep <- prep_recipe(analysis_set)
    analysis_processed <- bake(analysis_prep, new_data = analysis_set)
    #parsnip::xgb_train()
   model <- boost_tree(mode = "regression", mtry = mtry, trees = trees,
  min_n = min_n, tree_depth = tree_depth, learn_rate = learn_rate,
  loss_reduction = loss_reduction, sample_size = sample_size)%>%
      set_engine("xgboost") %>%
        fit(price ~ ., data = analysis_processed)
     #model <- rand_forest(mode = "regression",mtry = mtry, trees = trees) %>%
    #    set_engine("ranger", importance = 'impurity') %>%
     #   fit(price ~ ., data = analysis_processed)
 assessment_set <- assessment(split)
assessment_prep <- prep(prep_recipe(assessment_set), testing = assessment_set)
  assessment_processed <- bake(assessment_prep, new_data = assessment_set)
tibble::tibble("id" = id,
        "truth" = assessment_processed$price,
        "prediction" = unlist(predict(model, new_data = assessment_processed)))
}



subsample	- sample_size   proprtion
trees  _
tree_depth - max_depth   integer
learn_rate   -eta    proportion 
mtry   -colsample_bytree

los_reduction  -  gamma , proportion
min_n  -  min_child_weight, proprtion


In [0]:
%%R

xgb_tuning <- function(param){
    validation_data <-  validation_data
    mtry <- param[1]
    trees <- param[2]
    min_n <- param[3]
    tree_depth <- param[4]
    learn_rate <- param[5]
    loss_reduction <- param[6]
    sample_size  <- param[7]

    results <- purrr::map2_df(.x = validation_data$splits,
                       .y = validation_data$id,
                       ~my_xgb(mtry = mtry, trees = trees, 
                               min_n=min_n,tree_depth=tree_depth,
                               learn_rate=learn_rate, 
                       loss_reduction=loss_reduction,
                       sample_size=sample_size,
                               
                               split = .x, id = .y))

    results %>%
        group_by(id) %>%
        f_meas(truth, prediction) %>%
        summarise(mean_f_meas = mean(.estimate)) %>%
        pull
}



xgb_reg_tuning <- function(param){
    validation_data <-  validation_data
    mtry <- param[1]
    trees <- param[2]
    min_n <- param[3]
    tree_depth <- param[4]
    learn_rate <- param[5]
    loss_reduction <- param[6]
    sample_size  <- param[7]

    results <- purrr::map2_df(.x = validation_data$splits,
                       .y = validation_data$id,
                       ~my_xgb_reg(mtry = mtry, trees = trees, 
                               min_n=min_n,tree_depth=tree_depth,
                               learn_rate=learn_rate, 
                       loss_reduction=loss_reduction,
                       sample_size=sample_size,
                               
                               split = .x, id = .y))

    results %>%
        group_by(id) %>%
      yardstick::rmse(truth, prediction) %>%
        summarise(mean_rmse = mean(.estimate)) %>%
        pull
}




In [0]:

results_example <- map2_df(.x = validation_data$splits,
                           .y = validation_data$id,
                           ~my_xgb( mtry = 3, trees = 200,
                                    min_n=0.2,tree_depth=7,learn_rate=0.01,
                                   loss_reduction=0.01,sample_size=0.7,  
                                   split = .x, id = .y))
head(results_example)


I can now compute the RMSE when mtry = 3 and trees = 200:

In [0]:
%%R


Accuracy=results_example %>%
    group_by(id) %>%
    accuracy(truth, prediction) %>%
    summarise(mean_acc = mean(.estimate)) %>%
    pull



Precision = results_example %>%
    group_by(id) %>%
    precision(truth, prediction) %>%
    summarise(mean_prec = mean(.estimate)) %>%
    pull


Recall = results_example %>%
    group_by(id) %>%
    recall(truth, prediction) %>%
    summarise(mean_recall = mean(.estimate)) %>%
    pull

F_mea = results_example %>%
    group_by(id) %>%
    f_meas(truth, prediction) %>%
    summarise(mean_f = mean(.estimate)) %>%
    pull


Balanced_Accuracy = results_example %>%
    group_by(id) %>%
    bal_accuracy(truth, prediction) %>%
    summarise(mean_bal_acc = mean(.estimate)) %>%
    pull


# AUC= results_example %>%
#   #mutate(truth= as.double(truth)-1)%>%
#     group_by(id) %>%
#     roc_auc(truth, prediction) %>%
#     summarise(mean_bal_acc = mean(.estimate)) %>%
#     pull




library(glue)

#glue('AUC is {round(AUC,3)}.')
glue('F Meaure  is {round(F_mea,3)}.')
glue('Balanced_Accuracy is {round(Balanced_Accuracy,3)}.')
glue('Recall is {round(Recall,3)}.')
glue('Precision is {round(Precision,3)}.')
glue('Accuracy is {round(Accuracy,3)}.')


# predictions_glm %>%
#   conf_mat(price, .pred_class) %>%
#   pluck(1) %>%
#   as_tibble() %>%
#   ggplot(aes(Prediction, Truth, alpha = n)) +
#   geom_tile(show.legend = FALSE) +
#   geom_text(aes(label = n), colour = "white", alpha = 1, size = 8)
# 


In [0]:
%%R

cross_val_tbl <- vfold_cv(train_data, v = 10)
cross_val_tbl

cross_val_tbl$splits %>%
  pluck(1)


#### Grid Search Hyperparameter Tuning

Using Grid_data,the number of models to be trained 4*3*3*3 equal 108 different models. These comibations will take far too much time to run. For this reason we would limit ourselves to a smaller combination of hyperparameter values using grid_data.

mtry <- param[1]
    trees <- param[2]
    min_n <- param[3]
    tree_depth <- param[4]
    learn_rate <- param[5]
    loss_reduction <- param[6]
    sample_size  <- param[7]


In [0]:
%%R



Grid_data= expand.grid(mtry=4:7,trees=c(200,250,300),
                       learn_rate=c(0.001,0.01,0.1),tree_depth=c(6,7,8))

#metric_results=furrr::future_map2_dbl(.x=4:5,.y=200:201,~xgb_tuning(c(.x, .y,0.5,10,0.01,0.2,0.8)))





grid_data= tibble(mtry=3:10,trees=c(50,100,150,200,250,300,350,400))

metric_results=furrr::future_map2_dbl(.x=grid_data$mtry,
                                      .y=grid_data$trees,
                                      ~xgb_tuning(c(.x, .y,0.5,10,0.01,0.2,0.8), validation_data))




Three - D plot of mtry,trees and AUC

In [0]:
%%R

library(plotly)

mtcars$am[which(mtcars$am == 0)] <- 'Automatic'
mtcars$am[which(mtcars$am == 1)] <- 'Manual'
mtcars$am <- as.factor(mtcars$am)

p <- plot_ly(mtcars, x = ~wt, y = ~hp, z = ~qsec, color = ~am, colors = c('#BF382A', '#0C4B8E')) %>%
  add_markers() %>%
  layout(scene = list(xaxis = list(title = 'Weight'),
                     yaxis = list(title = 'Gross horsepower'),
                     zaxis = list(title = '1/4 mile time')))


Multi-paramter Grid Search

In [0]:
%%R


df=tibble(
  mtry=4:5,
  trees=200:201,
  min_n=c(0.5,0.6),
  tree_depth=9:10,
  learn_rate=c(0.01,0.02),
  loss_reduction=c(0.2,0.3),
  sample_size = c(0.7,0.8)
)



xgb_tuning2 <- function(mtry,trees,min_n,tree_depth,learn_rate,loss_reduction,sample_size){
    validation_data <-  validation_data
 

    results <- purrr::map2_df(.x = validation_data$splits,
                       .y = validation_data$id,
                       ~my_xgb(mtry = mtry, trees = trees, 
                               min_n=min_n,tree_depth=tree_depth,
                               learn_rate=learn_rate, 
                       loss_reduction=loss_reduction,
                       sample_size=sample_size,
                               
                               split = .x, id = .y))

  f_value <-  results %>%
        group_by(id) %>%
        f_meas(truth, prediction) %>%
        summarise(mean_f_meas = mean(.estimate)) %>%
        pull
    
    return(tibble(f_value=f_value))
}



metric_results=furrr::future_pmap_dfr(df, xgb_tuning2)


#metric_results= future.apply::future_mapply(xgb_tuning2, 
#df$mtry,df$trees,df$min_n,df$tree_depth,df$learn_rate,df$loss_reduction,df$sample_size)

